In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names.
student_data_df.head(10)

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [ ]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [ ]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[
    (student_data_df['school_name'] == "Thomas High School") &
    (student_data_df['grade'] == "9th"),
    ['reading_score']
] = np.nan
student_data_df.tail(10)

In [ ]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[
    (student_data_df['school_name'] == "Thomas High School") &
    (student_data_df['grade'] == "9th"),
    ['math_score']
] = np.nan
student_data_df.tail(10)

In [ ]:
#  Step 4. Check the student data for NaN's. 

student_data_df.isnull().sum()

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [ ]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

In [ ]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [ ]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [ ]:
student_data_df

In [ ]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
thomas_9th_graders = student_data_df.loc[
    (student_data_df['grade'] == '9th') &
    (student_data_df['school_name'] == 'Thomas High School')
].count()['Student ID']

# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()


# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
new_student_count = student_count - thomas_9th_graders
new_student_count

In [ ]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[
    (school_data_complete_df["math_score"] >= 70)
].count()["student_name"]

passing_reading_count = school_data_complete_df[
    (school_data_complete_df["reading_score"] >= 70)
].count()["student_name"]

In [ ]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = passing_math_count / new_student_count * 100
passing_reading_percentage = passing_reading_count / new_student_count * 100

In [ ]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[
    (school_data_complete_df["math_score"] >= 70) &
    (school_data_complete_df["reading_score"] >= 70)
]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()


# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = overall_passing_math_reading_count / new_student_count * 100

In [ ]:
# Create a DataFrame
district_summary_df = pd.DataFrame([
    {"Total Schools": school_count,
     "Total Students": student_count,
     "Total Budget": total_budget,
     "Average Math Score": average_math_score,
     "Average Reading Score": average_reading_score,
     "% Passing Math": passing_math_percentage,
     "% Passing Reading": passing_reading_percentage,
     "% Overall Passing": overall_passing_percentage
    }
])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

##  School Summary

In [ ]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [ ]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


# per_school_summary_df.head()

In [ ]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df

In [ ]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
thomas_10th_graders = student_data_df.loc[
    (student_data_df['grade'] == '10th') &
    (student_data_df['school_name'] == 'Thomas High School')
].count()['Student ID']
thomas_10th_graders

In [ ]:
thomas_11th_graders = student_data_df.loc[
    (student_data_df['grade'] == '11th') &
    (student_data_df['school_name'] == 'Thomas High School')
].count()['Student ID']
thomas_11th_graders

In [ ]:
thomas_12th_graders = student_data_df.loc[
    (student_data_df['grade'] == '12th') &
    (student_data_df['school_name'] == 'Thomas High School')
].count()['Student ID']
thomas_12th_graders

In [ ]:
# Step 6. Get all the students passing math from THS
thomas_passing_math = student_data_df.loc[
    (student_data_df['math_score'] >= 70) &
    (student_data_df['school_name'] == 'Thomas High School')
]
thomas_passing_math

In [ ]:
# Step 7. Get all the students passing reading from THS
thomas_passing_reading = student_data_df.loc[
    (student_data_df['reading_score'] >= 70) &
    (student_data_df['school_name'] == 'Thomas High School')
]
thomas_passing_reading

In [ ]:
# Step 8. Get all the students passing math and reading from THS
thomas_passing_both = student_data_df.loc[
    (student_data_df['math_score'] >= 70) &
    (student_data_df['reading_score'] >= 70) &
    (student_data_df['school_name'] == 'Thomas High School')
]
thomas_passing_both

In [ ]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 

# First get total number of students in Thomas Hight School from 10th-12th grade
thomas_student_count = thomas_10th_graders + thomas_11th_graders + thomas_12th_graders

thomas_math_pct = thomas_passing_math.count()['Student ID'] / thomas_student_count * 100
thomas_math_pct

In [ ]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
thomas_reading_pct = thomas_passing_reading.count()['Student ID'] / thomas_student_count * 100
thomas_reading_pct

In [ ]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
thomas_overall_pct = thomas_passing_both.count()['Student ID'] / thomas_student_count * 100
thomas_overall_pct

In [ ]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[['Thomas High School'],['% Passing Math']] = thomas_math_pct

In [ ]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[['Thomas High School'],['% Passing Reading']] = thomas_reading_pct

In [ ]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc[['Thomas High School'],['% Overall Passing']] = thomas_overall_pct

In [ ]:
per_school_summary_df

## High and Low Performing Schools 

In [ ]:
# Sort and show top five schools.
per_school_summary_df.sort_values(['% Overall Passing'], ascending = False).head()

In [ ]:
# Sort and show bottom five schools.
per_school_summary_df.sort_values(['% Overall Passing'], ascending = True).head()

## Math and Reading Scores by Grade

In [ ]:
# Create a Series of scores by grade levels using conditionals.
ninth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "9th")]

tenth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "10th")]

eleventh_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "11th")]

twelfth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "12th")]

# Group each school Series by the school name for the average math score.
ninth_math = ninth_grade_df.groupby(['school_name']).mean()['math_score']
tenth_math = tenth_grade_df.groupby(['school_name']).mean()['math_score']
eleventh_math = eleventh_grade_df.groupby(['school_name']).mean()['math_score']
twelfth_math = twelfth_grade_df.groupby(['school_name']).mean()['math_score']


# Group each school Series by the school name for the average reading score.
ninth_reading = ninth_grade_df.groupby(['school_name']).mean()['reading_score']
tenth_reading = tenth_grade_df.groupby(['school_name']).mean()['reading_score']
eleventh_reading = eleventh_grade_df.groupby(['school_name']).mean()['reading_score']
twelfth_reading = twelfth_grade_df.groupby(['school_name']).mean()['reading_score']


In [ ]:
# Combine each Series for average math scores by school into single data frame.
by_school_math = pd.DataFrame({
    '9th Grade': ninth_math,
    '10th Grade': tenth_math,
    '11th Grade': eleventh_math,
    '12th Grade': twelfth_math
})

In [ ]:
# Combine each Series for average reading scores by school into single data frame.
by_school_reading = pd.DataFrame({
    '9th Grade': ninth_reading,
    '10th Grade': tenth_reading,
    '11th Grade': eleventh_reading,
    '12th Grade': twelfth_reading
})

In [ ]:
# Format each grade column.


In [ ]:
# Remove the index.
by_school_math.index.name = None


# Display the data frame
by_school_math

In [ ]:
## Remove the index.
by_school_reading.index.name = None

# Display the data frame
by_school_reading

## Scores by School Spending

In [ ]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-$629", "$630-$644", "$645-$675"]

# Categorize spending based on the bins.


In [ ]:
# Add bin data to the per school summary
per_school_summary_df.head()

In [ ]:
per_school_summary_df['Spending Range Per Student'] = pd.cut(per_school_capita, spending_bins, labels = group_names)
per_school_summary_df

In [ ]:
# Calculate averages for the desired columns. 
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Range Per Student"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Range Per Student"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Range Per Student"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Range Per Student"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Range Per Student"]).mean()["% Overall Passing"]

In [ ]:
# Create the DataFrame
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

spending_summary_df

In [ ]:
# Format the DataFrame 
# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df

## Scores by School Size

In [64]:
# Establish the bins.
# set bins to Small, Medium, Large
size_bins = [0, 1000, 2000, 5000]
# Set bin labels
size_labels = ['< 1,000', '1,000-1,999', '2,000-5,000']
# Categorize spending based on the bins.
per_school_summary_df['Size Range'] = pd.cut(per_school_counts, size_bins, labels = size_labels)

In [66]:
# Calculate averages for the desired columns. 
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["Size Range"]).mean()["Average Math Score"]

size_reading_scores = per_school_summary_df.groupby(["Size Range"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["Size Range"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["Size Range"]).mean()["% Passing Reading"]

overall_passing_size = per_school_summary_df.groupby(["Size Range"]).mean()["% Overall Passing"]

In [68]:
# Assemble into DataFrame. 
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": overall_passing_size})

In [69]:
# Format the DataFrame  
# Formatting
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Size Range,,,,,
"< 1,000",83.8,83.9,94,96,90
"1,000-1,999",83.4,83.9,94,97,91
"2,000-5,000",77.7,81.3,70,83,58


## Scores by School Type

In [70]:
# Calculate averages for the desired columns. 
# Create a new dataframe to hold the averages by school type
type_summary_df = per_school_summary_df.groupby(['School Type']).mean()
type_summary_df

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,,
Charter,1524.250000,83.465425,83.902315,93.610020,96.550223,90.392533
District,3853.714286,76.956733,80.966636,66.548453,80.799062,53.672208


In [71]:
# Assemble into DataFrame. 


In [72]:
# # Format the DataFrame 
# Add Formatting
# Formatting
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df


,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,,
Charter,1524.250000,83.5,83.9,94,97,90
District,3853.714286,77.0,81.0,67,81,54


## Analysis including Thomas High School 9th Grade Scores

In [ ]:
# Add the Pandas dependency
import pandas as pd

In [ ]:
# Files to load
schools_file = "Resources/schools_complete.csv"
students_file = "Resources/students_complete.csv"

In [ ]:
# Read the school data file and store it in a Pandas DataFrame
school_data_df = pd.read_csv(schools_file)
school_data_df.head()

In [ ]:
# Read the students data file and store it in a Pandas DataFrame
student_data_df = pd.read_csv(students_file)
student_data_df.head()

In [ ]:
# Determine if there are any missing values in the school data
school_data_df.count()

In [ ]:
# Determine if there are any missing values in the student data
student_data_df.count()

In [ ]:
# Determine if there are any missing values in the school data
school_data_df.isnull()

In [ ]:
# Determine if there are any missing values in the student data
student_data_df.isnull().sum()

In [ ]:
# Determine if there are *not* any missing values in the school data
school_data_df.notnull()

In [ ]:
# Determine if there are *not* any missing values in the student data
student_data_df.notnull().sum()

In [ ]:
# Determine the data type of each column in school_data_df
school_data_df.dtypes

In [ ]:
# Find the data type for the column budget only
school_data_df.budget.dtype
school_data_df["budget"].dtype

In [ ]:
# Determine the data type of each column in student_data_df
student_data_df.dtypes

In [ ]:
# Use results from cleaning_student_names.ipynb to clean the data
# Define the list of prefixes and suffixes to replace
prefix_suffix = ['Dr. ', 'Miss ', 'Mr. ', 'Mrs. ', 'Ms. ', ' DDS', ' DVM', ' MD', ' PhD']
for ps in prefix_suffix:
    student_data_df.student_name = student_data_df.student_name.str.replace(ps,'')

In [ ]:
# Define a new DataFrame by merging the two datasets
school_data_complete_df = pd.merge(
    student_data_df,
    school_data_df,
    on=["school_name", "school_name"]
)
school_data_complete_df.head()

In [ ]:
# Check the count of each column
student_count = school_data_complete_df.count()
student_count

In [ ]:
# Define the number of students
student_count = school_data_complete_df['Student ID'].count()
student_count

In [ ]:
# Get the total number of schools
school_count = school_data_complete_df.school_name.unique()
school_count = len(school_count)
school_count

In [ ]:
# Get the total budget of the school district
district_budget = school_data_df.budget.sum()
district_budget

In [ ]:
# Get the average scores for reading and math
reading_avg = school_data_complete_df.reading_score.mean()
reading_avg

In [ ]:
# Get the average scores for reading and math
math_avg = school_data_complete_df.math_score.mean()
math_avg

In [ ]:
# Determine passing percentages in reading, math, and overall
# Passing score is 70
passing_score = 70

# New dataframe where math score is greater or equal to passing_score
# Returns a list of boolean values
passing_math = school_data_complete_df.math_score >= passing_score
passing_reading = school_data_complete_df.reading_score >= passing_score

In [ ]:
# Get the number of students who passed math
passing_math_total = passing_math.sum()
passing_math_total

In [ ]:
# Get the number of students who pass reading
passing_reading_total = passing_reading.sum()
passing_reading_total

In [ ]:
# To filter the dataframe based on the passing score
passing_math_df = school_data_complete_df[school_data_complete_df.math_score >= passing_score]
passing_math_df.count()

In [ ]:
# To filter the dataframe based on the passing score
passing_reading_df = school_data_complete_df[school_data_complete_df.reading_score >= passing_score]
passing_reading_df.count()

In [ ]:
# Get the Percentages of students who passed math and reading respectively
math_pct = passing_math_total / student_count * 100
reading_pct = passing_reading_total / student_count * 100
print(f"The pct of students who passed math is {math_pct:.2f}%")
print(f"The pct of students who passed reading is {reading_pct:.2f}%")

In [ ]:
# Get the dataframe of students who passed both math and reading
passing_both_df = school_data_complete_df[
    (school_data_complete_df.math_score >= passing_score) &
    (school_data_complete_df.reading_score >= passing_score)
]
passing_both_df.head()

In [ ]:
# Get the total number of students who passed both math and reading
passing_both_total = passing_both_df.student_name.count()
passing_both_total

In [ ]:
# Get the percentage of students who passed both math and reading
both_pct = passing_both_total / student_count * 100
print(f"The pct of students who passed both math and reading is {both_pct:.2f}%")

In [ ]:
# Add summary statistics into one dataframe
district_summary_df = pd.DataFrame(
    [
        {
            "Total Schools": school_count,
            "Total Students": student_count,
            "Total Budget": district_budget,
            "Average Math Score": math_avg,
            "Average Reading Score": reading_avg,
            "% Passing Math": math_pct,
            "% Passing Reading": reading_pct,
            "% Overall Passing": both_pct
        }
    ]
)
district_summary_df

In [ ]:
# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

district_summary_df["Total Students"]

In [ ]:
# Format the "Total Budget" to have the comma for a thousands separator and dollar sign
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df["Total Budget"]

In [ ]:
# Format the average scores to 1 decimal
# .. and pct passing to the nearest whole number
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["Average Math Score"]

In [ ]:
# Format the average scores to 1 decimal
# .. and pct passing to the nearest whole number
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["Average Reading Score"]

In [ ]:
# Format the average scores to 1 decimal
# .. and pct passing to the nearest whole number
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)

district_summary_df["% Passing Math"]

In [ ]:
# Format the average scores to 1 decimal
# .. and pct passing to the nearest whole number
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)

district_summary_df["% Passing Reading"]

In [ ]:
# Format the average scores to 1 decimal
# .. and pct passing to the nearest whole number
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)

district_summary_df["% Overall Passing"]

In [ ]:
# Check on the updated dataframe with formatting
district_summary_df

In [ ]:
# Create a new DataFrame to hold the school names
# Use the existing school_data_df
per_school_types = school_data_df.set_index(['school_name'])['type']
per_school_types

In [ ]:
# Convert the Series into a DataFrame
per_school_df = pd.DataFrame(per_school_types)
per_school_df

In [ ]:
# Get a new series with index as school name, and values as school size
per_school_size = school_data_df.set_index(['school_name'])['size']
per_school_size

In [ ]:
# Get a new series with index as school name, and
# .. values as number of students from the test results df school_data_complete_df

per_school_counts = school_data_complete_df['school_name'].value_counts()
per_school_counts


In [ ]:
sum(per_school_size)

In [ ]:
sum(per_school_counts)

In [ ]:
# Both methods gets the same number of students

# Add the budget per student per each school
per_school_budget = school_data_df.set_index(['school_name'])['budget']
per_school_budget

In [ ]:
# Calculate the per student spending
budget_per_student = per_school_budget / per_school_counts
budget_per_student

In [ ]:
# First get a list of math scores with the school as the index
student_school_math = student_data_df.set_index(['school_name'])['math_score']
student_school_math

In [ ]:
# Calculate the average of each column in school_data_complete_df
per_school_averages = school_data_complete_df.groupby(
    ['school_name']
).mean()
per_school_averages

In [ ]:
# Use the averages dataframe and only get math and reading scores
per_school_math = per_school_averages['math_score']
per_school_math

In [ ]:
per_school_reading = per_school_averages['reading_score']
per_school_reading

In [ ]:
# Calculate the passing scores by creating a filtered DataFrame
per_school_passing_math = school_data_complete_df[
    (school_data_complete_df['math_score'] >= passing_score)]
per_school_passing_math

In [ ]:
per_school_passing_reading = school_data_complete_df[
    (school_data_complete_df['reading_score'] >= passing_score)
]
per_school_passing_reading

In [ ]:
# Group the passing math results by school
per_school_passing_math = per_school_passing_math.groupby(
    ['school_name']).count()['student_name']
per_school_passing_math

In [ ]:
# Group the passing reading results by school
per_school_passing_reading = per_school_passing_reading.groupby(
    ['school_name']).count()['student_name']
per_school_passing_reading

In [ ]:
# Calculate the pct of passing math and reading scores
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_math

In [ ]:
# Calculate the pct of passing math and reading scores
per_school_passing_reading = per_school_passing_reading / per_school_counts *100
per_school_passing_reading

In [ ]:
# Get students who passed both math and reading
per_school_passing_both = school_data_complete_df[
    (school_data_complete_df['math_score'] >= passing_score) &
    (school_data_complete_df['reading_score'] >= passing_score)
]
per_school_passing_both

In [ ]:
# Group by school
per_school_passing_both = per_school_passing_both.groupby(
    ['school_name']).count()['student_name']
per_school_passing_both

In [ ]:
# Get the pct for each school for students who passed both
per_school_passing_both = per_school_passing_both / per_school_counts * 100
per_school_passing_both

In [ ]:
# Combine data into one dataframe
per_school_df = pd.DataFrame({
    'School Type': per_school_types,
    'Total Students': per_school_counts,
    'Total School Budget': per_school_budget,
    'Per Student Budget': budget_per_student,
    'Average Math Score': per_school_math,
    'Average Reading Score': per_school_reading,
    '% Passing Math': per_school_passing_math,
    '% Passing Reading': per_school_passing_reading,
    '% Overall Passing': per_school_passing_both
})
per_school_df.head()

In [ ]:
# Format the Total School Budget and Per Student Budget columns
per_school_df['Total School Budget'] = per_school_df[
    'Total School Budget'].map("${:,.2f}".format)

In [ ]:
per_school_df['Per Student Budget'] = per_school_df[
    'Per Student Budget'].map("${:,.2f}".format)

In [ ]:
# Check out the formatting done so far
per_school_df.head()

In [ ]:
# Sort the dataframe based on % Overall Passing
per_school_df = per_school_df.sort_values(['% Overall Passing'], ascending = False)

In [ ]:
# Display the new dataframe
per_school_df

In [ ]:
# After sorting, the top five schools are the head() and bottom five schools are the tail()
top_five_schools = per_school_df.head()
bottom_five_schools = per_school_df.tail()

In [ ]:
# Sort the bottom five schools in ascending order
bottom_five_schools = bottom_five_schools.sort_values(['% Overall Passing'], ascending = True)

In [ ]:
# Work on grouping the average math and reading scores by school, by grade
# The index will be the school_name
school_data_complete_df.head()

In [ ]:
# Filter for the ninth grade first
ninth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "9th")]
ninth_grade_df.head()

In [ ]:
# Do the same for the other grades

tenth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "10th")]
tenth_grade_df.head()



In [ ]:
eleventh_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "11th")]
eleventh_grade_df.head()



In [ ]:
twelfth_grade_df = school_data_complete_df[
    (school_data_complete_df['grade'] == "12th")]
twelfth_grade_df.head()

In [ ]:
# Get the average math scores for each grade, grouped by school
ninth_grade_math = ninth_grade_df.groupby(['school_name']).mean()['math_score']
ninth_grade_math

In [ ]:
tenth_grade_math = tenth_grade_df.groupby(['school_name']).mean()['math_score']
tenth_grade_math

In [ ]:
eleventh_grade_math = eleventh_grade_df.groupby(['school_name']).mean()['math_score']
eleventh_grade_math

In [ ]:
twelfth_grade_math = twelfth_grade_df.groupby(['school_name']).mean()['math_score']
twelfth_grade_math

In [ ]:
# Get the average reading scores for each grade, grouped by school
ninth_grade_reading = ninth_grade_df.groupby(['school_name']).mean()['reading_score']
tenth_grade_reading = tenth_grade_df.groupby(['school_name']).mean()['reading_score']
eleventh_grade_reading = eleventh_grade_df.groupby(['school_name']).mean()['reading_score']
twelfth_grade_reading = twelfth_grade_df.groupby(['school_name']).mean()['reading_score']

In [ ]:
twelfth_grade_reading

In [ ]:
# Combine the series into one dataframe for math scores
school_grade_math = pd.DataFrame({
    '9th': ninth_grade_math,
    '10th': tenth_grade_math,
    '11th': eleventh_grade_math,
    '12th': twelfth_grade_math
})
school_grade_math

In [ ]:
# Combine the series into one dataframe for reading scores
school_grade_reading = pd.DataFrame({
    '9th': ninth_grade_reading,
    '10th': tenth_grade_reading,
    '11th': eleventh_grade_reading,
    '12th': twelfth_grade_reading
})
school_grade_reading

In [ ]:
# Format each of the reading and math average by school dataframes
school_grade_math["9th"] = school_grade_math["9th"].map("{:.1f}".format)

In [ ]:
school_grade_math

In [ ]:
school_grade_math["10th"] = school_grade_math["10th"].map("{:.1f}".format)
school_grade_math["11th"] = school_grade_math["11th"].map("{:.1f}".format)
school_grade_math["12th"] = school_grade_math["12th"].map("{:.1f}".format)

In [ ]:
school_grade_reading["9th"] = school_grade_reading["9th"].map("{:.1f}".format)
school_grade_reading["10th"] = school_grade_reading["10th"].map("{:.1f}".format)
school_grade_reading["11th"] = school_grade_reading["11th"].map("{:.1f}".format)
school_grade_reading["12th"] = school_grade_reading["12th"].map("{:.1f}".format)

In [ ]:
# Remove the index name for both dataframes
school_grade_math.index.name = None
school_grade_reading.index.name = None

In [ ]:
# Explore relationship between school budget and student performance
budget_per_student

In [ ]:
# Group the budget per student into bins
budget_per_student.describe()

In [ ]:
# Increase the bins by roughly the standard deviation, starting with a bin that captures more than one school
spending_bins = [0, 585, 585+30, 585+30*2, 585+30*3]
spending_bins

In [ ]:
# Use the cut() function to split the data into the determined bins
# Group by the bins, and count the number of schools in each bin
budget_per_student.groupby(pd.cut(budget_per_student, spending_bins)).count()

In [ ]:
# Regroup the bins so the number of schools in each bin is closer

In [ ]:
spending_bins = [0, 585, 630, 645, 675]
budget_per_student.groupby(pd.cut(budget_per_student, spending_bins)).count()

In [ ]:
# Define the name of groups
group_names = ['<$584', '$585-$629', '$630-$644', '$645-$675']

In [ ]:
# Add bin data to the per school summary
per_school_df.head()

In [ ]:
per_school_df['Spending Range Per Student'] = pd.cut(budget_per_student, spending_bins, labels = group_names)
per_school_df

In [ ]:
# Get the average for each group for math, reading, pct passing path, pct passing reading, and overall passing pct

In [ ]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_df.groupby(["Spending Range Per Student"]).mean()["Average Math Score"]

spending_reading_scores = per_school_df.groupby(["Spending Range Per Student"]).mean()["Average Reading Score"]

spending_passing_math = per_school_df.groupby(["Spending Range Per Student"]).mean()["% Passing Math"]

spending_passing_reading = per_school_df.groupby(["Spending Range Per Student"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_df.groupby(["Spending Range Per Student"]).mean()["% Overall Passing"]

In [ ]:
overall_passing_spending

In [ ]:
(spending_passing_math + spending_passing_reading)/2

In [ ]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

spending_summary_df

In [ ]:
# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df

In [ ]:
# Get bins for school size
per_school_size

In [ ]:
per_school_size.describe()

In [ ]:
# set bins to Small, Medium, Large
size_bins = [0, 1000, 2000, 5000]

In [ ]:
per_school_size.groupby(pd.cut(per_school_size, size_bins)).count()

In [ ]:
# Set bin labels
size_labels = ['< 1,000', '1,000-1,999', '2,000-5,000']

In [ ]:
per_school_df['Size Range'] = pd.cut(per_school_size, size_bins, labels = size_labels)
per_school_df

In [ ]:
# Calculate averages for the desired columns.
size_math_scores = per_school_df.groupby(["Size Range"]).mean()["Average Math Score"]

size_reading_scores = per_school_df.groupby(["Size Range"]).mean()["Average Reading Score"]

size_passing_math = per_school_df.groupby(["Size Range"]).mean()["% Passing Math"]

size_passing_reading = per_school_df.groupby(["Size Range"]).mean()["% Passing Reading"]

overall_passing_size = per_school_df.groupby(["Size Range"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": overall_passing_size})

size_summary_df.dtypes

In [ ]:
# Formatting
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

In [ ]:
# Create a new dataframe to hold the averages by school type
type_summary_df = per_school_df.groupby(['School Type']).mean()
type_summary_df

In [ ]:
# Add Formatting
# Formatting
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df